In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress
import sqlite3
from sqlalchemy.orm import Session
#  Data files
weather = "WeatherEvents_Jan2016-Dec2020[1].csv"
# Read the mouse data and the study results
weather_pd = pd.read_csv(weather)

# Display the data table for preview
weather_pd

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6274201,W-6276144,Snow,Light,2020-12-14 17:30:00,2020-12-15 01:53:00,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0
6274202,W-6276145,Snow,Moderate,2020-12-15 01:53:00,2020-12-15 03:19:00,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0
6274203,W-6276146,Snow,Light,2020-12-15 03:19:00,2020-12-15 05:53:00,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0
6274204,W-6276147,Snow,Light,2020-12-23 04:24:00,2020-12-23 05:53:00,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0


In [4]:
data = weather_pd.loc[:, ["Type", "Severity","StartTime(UTC)","EndTime(UTC)","LocationLat","LocationLng","City","County","State","ZipCode"]]
data

,Type,Severity,StartTime(UTC),EndTime(UTC),LocationLat,LocationLng,City,County,State,ZipCode
0,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
...,...,...,...,...,...,...,...,...,...,...
6274201,Snow,Light,2020-12-14 17:30:00,2020-12-15 01:53:00,42.5833,-108.2833,Lander,Fremont,WY,82520.0
6274202,Snow,Moderate,2020-12-15 01:53:00,2020-12-15 03:19:00,42.5833,-108.2833,Lander,Fremont,WY,82520.0
6274203,Snow,Light,2020-12-15 03:19:00,2020-12-15 05:53:00,42.5833,-108.2833,Lander,Fremont,WY,82520.0
6274204,Snow,Light,2020-12-23 04:24:00,2020-12-23 05:53:00,42.5833,-108.2833,Lander,Fremont,WY,82520.0


In [14]:
Weather_Results = data.loc[data['State'] == "NY"]
Weather_Results = Weather_Results.dropna()

In [15]:
Weather_Results 

,Type,Severity,StartTime(UTC),EndTime(UTC),LocationLat,LocationLng,City,County,State,ZipCode
91852,Cold,Severe,2016-01-07 13:55:00,2016-01-07 14:15:00,42.1534,-79.2580,Jamestown,Chautauqua,NY,14701.0
91853,Cold,Severe,2016-01-20 17:35:00,2016-01-20 17:55:00,42.1534,-79.2580,Jamestown,Chautauqua,NY,14701.0
91854,Cold,Severe,2016-01-22 17:35:00,2016-01-22 17:55:00,42.1534,-79.2580,Jamestown,Chautauqua,NY,14701.0
91855,Cold,Severe,2016-01-22 18:15:00,2016-01-22 18:35:00,42.1534,-79.2580,Jamestown,Chautauqua,NY,14701.0
91856,Cold,Severe,2016-02-16 10:55:00,2016-02-16 11:15:00,42.1534,-79.2580,Jamestown,Chautauqua,NY,14701.0
...,...,...,...,...,...,...,...,...,...,...
5863189,Snow,Light,2020-12-29 01:53:00,2020-12-29 02:53:00,42.1599,-76.8916,Big Flats,Chemung,NY,14845.0
5863190,Snow,Light,2020-12-29 10:53:00,2020-12-29 11:40:00,42.1599,-76.8916,Big Flats,Chemung,NY,14845.0
5863191,Snow,Light,2020-12-29 16:53:00,2020-12-29 17:53:00,42.1599,-76.8916,Big Flats,Chemung,NY,14845.0
5863192,Rain,Light,2020-12-31 01:53:00,2020-12-31 02:53:00,42.1599,-76.8916,Big Flats,Chemung,NY,14845.0


In [23]:
Weather_Results.to_csv("Weather_Results.csv",index=False)

In [24]:
Results = pd.read_csv("Weather_Results.csv")
Results

,Type,Severity,StartTime(UTC),EndTime(UTC),LocationLat,LocationLng,City,County,State,ZipCode
0,Cold,Severe,2016-01-07 13:55:00,2016-01-07 14:15:00,42.1534,-79.2580,Jamestown,Chautauqua,NY,14701.0
1,Cold,Severe,2016-01-20 17:35:00,2016-01-20 17:55:00,42.1534,-79.2580,Jamestown,Chautauqua,NY,14701.0
2,Cold,Severe,2016-01-22 17:35:00,2016-01-22 17:55:00,42.1534,-79.2580,Jamestown,Chautauqua,NY,14701.0
3,Cold,Severe,2016-01-22 18:15:00,2016-01-22 18:35:00,42.1534,-79.2580,Jamestown,Chautauqua,NY,14701.0
4,Cold,Severe,2016-02-16 10:55:00,2016-02-16 11:15:00,42.1534,-79.2580,Jamestown,Chautauqua,NY,14701.0
...,...,...,...,...,...,...,...,...,...,...
153381,Snow,Light,2020-12-29 01:53:00,2020-12-29 02:53:00,42.1599,-76.8916,Big Flats,Chemung,NY,14845.0
153382,Snow,Light,2020-12-29 10:53:00,2020-12-29 11:40:00,42.1599,-76.8916,Big Flats,Chemung,NY,14845.0
153383,Snow,Light,2020-12-29 16:53:00,2020-12-29 17:53:00,42.1599,-76.8916,Big Flats,Chemung,NY,14845.0
153384,Rain,Light,2020-12-31 01:53:00,2020-12-31 02:53:00,42.1599,-76.8916,Big Flats,Chemung,NY,14845.0
